In [1]:
import torch
import numpy as np

In [41]:
num_samples = 10
input_size = 16
num_fillings = [8, 8]

samples = torch.zeros(num_samples, 2 * input_size, dtype=torch.int)
# The first num_fillings[0] represents the filled sites for spin up
positions_up = torch.randperm(input_size)
# The next num_fillings[1] represents the filled sites for spin down
positions_down = torch.randperm(input_size)
prob_old = 1
for i in range(num_samples):
    # Metropolis-Hastings
    ## Randomly select n out of L occupied sites and n unoccupied sites to swap.
    ## As the number of up spins and down spins are converved separately,
    ## we swap the occupied and unoccupied sites for each spin separately.
    n = 1
    ### For spin up
    indices_up_occupied = torch.randperm(num_fillings[0])[:n]
    indices_up_unoccupied = torch.randperm(input_size - num_fillings[0])[:n] + num_fillings[0]           
    elements_up_occupied = positions_up[indices_up_occupied]
    elements_up_unoccupied = positions_up[indices_up_unoccupied]
    ### For spin down
    indices_down_occupied = torch.randperm(num_fillings[1])[:n]
    indices_down_unoccupied = torch.randperm(input_size - num_fillings[1])[:n] + num_fillings[1]           
    elements_down_occupied = positions_down[indices_down_occupied]
    elements_down_unoccupied = positions_down[indices_down_unoccupied]
    ## Clone the occupied and unoccupied sites, and swap the elements
    new_positions_up = positions_up.clone()
    new_positions_up[indices_up_occupied] = elements_up_unoccupied
    new_positions_up[indices_up_unoccupied] = elements_up_occupied

    new_positions_down = positions_down.clone()
    new_positions_down[indices_down_occupied] = elements_down_unoccupied
    new_positions_down[indices_down_unoccupied] = elements_down_occupied
    # Create the new configuration
    config = torch.ones(2 * input_size, dtype=torch.float)
    config[new_positions_up[num_fillings[0]:]] = -1
    config[new_positions_down[num_fillings[1]:] + input_size] = -1
    #print(sum(config))


    #out_config_up, out_config_down = forward(config.unsqueeze(0))
    out_config_up = torch.rand(1, input_size, num_fillings[0])
    out_config_down = torch.rand(1, input_size, num_fillings[1])
    
    # Calculate the acceptance probability
    overlap_up = torch.det(out_config_up[:, new_positions_up[:num_fillings[0]], :])
    overlap_down = torch.det(out_config_down[:, new_positions_down[:num_fillings[1]], :])
    overlap = torch.abs(overlap_up * overlap_down) ** 2

    prob_new = overlap[0].item()
    if i == 0:
        prob_old = prob_new
    # Accept or reject the new configuration
    if torch.rand(1).item() < prob_new / prob_old:
        new_positions_up = positions_up
        new_positions_down = positions_down
        prob_old = prob_new
        samples[i, :] = config
    else:
        samples[i, :] = samples[i-1, :]

In [42]:
torch.sum(samples,1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
a = torch.rand(4,3,3)
b = torch.zeros(4,3)
c = torch.nonzero(b, as_tuple=True)
c[1].shape
torch.det(a[:,c[1],:])

RuntimeError: linalg.det: A must be batches of square matrices, but they are 0 by 3 matrices

In [47]:
a = torch.zeros(3,6)
a[0,torch.randperm(6)[:3]] = 1
a[1,torch.randperm(6)[:3]] = 1
a[2,torch.randperm(6)[:3]] = 1
#print(a)
true_indices = torch.nonzero(a)
print(true_indices)
num_fillings = len(true_indices)
b = torch.zeros(a.size(0), a.size(1), num_fillings//a.size(0))
last_dim_indices = (torch.arange(num_fillings) % (num_fillings//a.size(0)))

b[true_indices[:, 0], true_indices[:, 1], last_dim_indices] = 1
#b[true_indices[:, 0],true_indices[:, 1], torch.tensor([0,1,2,0,1,2])] = 1
print(b)

tensor([[0, 0],
        [0, 3],
        [0, 5],
        [1, 2],
        [1, 4],
        [1, 5],
        [2, 0],
        [2, 2],
        [2, 5]])
tensor([[[1., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 1., 0.],
         [0., 0., 0.],
         [0., 0., 1.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [1., 0., 0.],
         [0., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 0., 0.],
         [0., 1., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 1.]]])


In [25]:
b = torch.zeros(a.size(0), a.size(1), 3)

# 生成最后一维的索引
last_dim_indices = (torch.arange(num_fillings) % 3)[:num_fillings]  # 形状 (num_fillings,)
last_dim_indices = last_dim_indices.unsqueeze(1)  # 增加一个维度以便广播

# 使用 scatter_ 填充
b[true_indices[:, 0], true_indices[:, 1], last_dim_indices.squeeze()] = 1
b

tensor([[[0., 0., 0.],
         [1., 0., 0.],
         [0., 0., 0.],
         [0., 1., 0.],
         [0., 0., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 0., 0.],
         [0., 0., 0.]]])